In [ ]:
import torch
import numpy as np
from PIL import Image
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.nn import functional as F
from sklearn.utils import resample
from torchvision.transforms import ToTensor
from torchvision import datasets, models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
patch_size = {patch_size}
patch_dir = 'PATCH-TO-PATCH_DIRECTORY' 

**LOADING PATIENTS' DATA** 

In [ ]:
# reading patients' data to obtain ground labels 

clinical_df = pd.read_csv('PATH-TO-CLINIDAL-DATA-CSV')
clinical_df = clinical_df[['Patient ID', 'Biomarker']]
clinical_df['Biomarker'] = clinical_df['Biomarker'].apply(lambda x: 1 if x == 'Positive' else 0)

clinical_df.head(3) 

**LOADING PATCH DATA**

In [ ]:
# reading patch data

patch_info = pd.read_csv("PATH-TO-PATCH-INFO-DATA-CSV")
patch_info.head(3) 

In [ ]:
df = pd.merge(patch_info, clinical_df, left_on = 'WSI No.', right_on = 'Patient ID', how = 'inner') 
df['Label'] = df['Biomarker'] 

df.head(3) 

In [ ]:
df = df[df['Annotation Id'] != -1]
df['Annotation Id'] = df['Annotation Id'] + 1 
df = df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"]) 

In [ ]:
# obtaining patch paths for the rest 

def obtain_patch_path(row): 
    
    wsi = str(row['WSI No.'])
    x = str(row['X Pixel'])
    y = str(row['Y Pixel'])
    
    return f'{patch_dir}/{wsi}_{x}_{y}.jpg'

df['Patch Path'] = df.apply(obtain_patch_path, axis=1) 
df.head(3) 

In [ ]:
print(len(df))

**DATASET SPLIT** 

In [ ]:
# reading train, val & test raw patches information into dataframes 

train_ids = ["List of Train WSI Ids"]
val_ids = ["List of Val WSI Ids"]
test_ids = ["List of Test WSI Ids"]

train_df = df[df['WSI No.'].isin(train_ids)] 
val_df = df[df['WSI No.'].isin(val_ids)] 
test_df = df[df['WSI No.'].isin(test_ids)] 

In [ ]:
train_wsi_nos = train_df['WSI No.'].to_list() 
train_ann_nos = train_df['Embedding No.'].to_list() 
train_labels_ = train_df['Label'].to_list() 
train_patch_paths = train_df['Patch Path'].to_list() 

val_wsi_nos = val_df['WSI No.'].to_list() 
val_ann_nos = val_df['Embedding No.'].to_list() 
val_labels_ = val_df['Label'].to_list() 
val_patch_paths = val_df['Patch Path'].to_list() 

test_wsi_nos = test_df['WSI No.'].to_list() 
test_ann_nos = test_df['Embedding No.'].to_list() 
test_labels_ = test_df['Label'].to_list() 
test_patch_paths = test_df['Patch Path'].to_list() 

**CLASS DISTRIBUTION** 

In [ ]:
def plot_patch_labels(ax, title, labels):
    ax.bar(['Biomarker (-)', 'Biomarker (+)'], [labels.count(0), labels.count(1)])
    ax.set_title(title)
    ax.set_ylabel('Count')

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

plot_bag_labels(axs[0], 'TRAIN PATCH LABELS', train_patch_labels)
plot_bag_labels(axs[1], 'VALIDATION PATCH LABELS', val_patch_labels)
plot_bag_labels(axs[2], 'TEST PATCH LABELS', test_patch_labels)

plt.tight_layout()
plt.show() 

**CUSTOM DATASETS & DATA LOADERS** 

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, patches, labels):
        assert len(patches) == len(labels), "Mismatch in number of patches and labels"
        self.patches = patches
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Check for valid index range
        if idx >= len(self.labels) or idx >= len(self.patches):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self)}")

        patch = Image.open(self.patches[idx]).convert("RGB") 
        patch = patch.resize((224, 224))  
        patch = ToTensor()(patch)
        patch = nn.functional.normalize(patch, dim=0, p=2)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return patch, label 

In [ ]:
# Create instances of custom datasets

train_dataset = CustomDataset(train_patch_paths, train_patch_labels)
valid_dataset = CustomDataset(val_patch_paths, val_patch_labels)
test_dataset = CustomDataset(test_patch_paths, test_patch_labels) 

In [ ]:
batch_size = {BATCH-SIZE}
num_of_workers = {NUMBER-OF-WORKERS}

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = num_of_workers)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers = num_of_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = num_of_workers)

**FINAL MODEL** 

In [ ]:
class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
          
        self.base = models.{'MODEL_NAME'}(weights=True)

        # Unfreeze model weights
        for param in self.base.parameters():
            param.requires_grad = True
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
    
    def forward(self,X):
        X = self.base(X)
        X = self.flatten(X)
        X = self.head(X)
        return X, F.sigmoid(X)

pos_weight = torch.tensor([POSITIVE-WEIGHT])
pos_weight = pos_weight.to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
model = Pipeline(1)

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    model = model.to('cuda')
    print('available')
    
optimizer = optim.Adam(params=model.parameters(), lr={LEARNING-RATE}) 

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Initialize empty lists to store loss and accuracy for training and validation 

train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []
roc_values_train = []
roc_values_val = []

In [ ]:
# Training loop

num_epochs = {NUMBER-OF-EPOCHS}

for epoch in range(num_epochs):
    model.train()
    y_true_train = []
    y_scores_train = []
    train_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  # L+=l.item()
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true_train.extend(labels.cpu().detach().numpy())
        y_scores_train.extend(outputs_with_sigmoid.cpu().detach().numpy())

    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
    roc_auc_train = auc(fpr_train, tpr_train)
    roc_values_train.append(roc_auc_train)

    # Validate your model after each epoch if needed
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    y_true_val = []
    y_scores_val = []
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            valid_loss += loss.item()  # L+=l.item()
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_true_val.extend(labels.cpu().detach().numpy())
            y_scores_val.extend(outputs_with_sigmoid.cpu().detach().numpy())

    valid_losses.append(valid_loss / len(valid_loader))
    valid_accuracies.append(100 * correct / total)

    fpr_val, tpr_val, _ = roc_curve(y_true_val, y_scores_val)
    roc_auc_val = auc(fpr_val, tpr_val)
    roc_values_val.append(roc_auc_val)

    # Classification report
    print(f'Epoch {epoch + 1}, Train Accuracy: {train_accuracies[-1]:.2f}%, Train Loss: {train_losses[-1]:.2f}%, Train ROC-AUC: {roc_auc_train:.2f}%, Val Accuracy: {valid_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.2f}%, Val ROC-AUC: {roc_auc_val:.2f}%')

    y_true_val = np.array(y_true_val)
    y_pred_val = (np.array(y_scores_val) >= 0.5).astype(int)
    print("Validation Classification Report:")
    print(classification_report(y_true_val, y_pred_val))

In [ ]:
def display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, subplot):
    if subplot % 10 == 1:  # set up the subplots on the first call
        plt.subplots(figsize=(12, 5), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_losses)
    ax.plot(valid_losses)
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid'])

    ax = plt.subplot(subplot + 1)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_accuracies)
    ax.plot(valid_accuracies)
    ax.set_title('Model Accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid'])

In [ ]:
# Plot the training curves 

display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, 121) 

**EVALUATING** 

In [ ]:
model.eval()
test_loss = 0
correct = 0
total = 0
y_true_test = []
y_pred_test = []
y_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        test_loss += loss.item()  # L+=l.item()
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true_test.extend(labels.cpu().detach().numpy())
        y_pred_test.extend(predicted.cpu().detach().numpy())
        y_probs.extend(outputs_with_sigmoid.cpu().detach().numpy())

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

# Calculate ROC-AUC for test set
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs)
roc_auc_test = auc(fpr_test, tpr_test)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.2f}%')

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_true_test, y_pred_test))

In [ ]:
# ROC curve for the test set
plt.figure(figsize=(4, 4))
plt.plot(fpr_test, tpr_test, label=f'Test ROC Curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
plt.legend()
plt.show()

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true_test, y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Bag Level Confusion Matrix')
plt.show()

**AGGREGATING PATCH LEVEL RESULTS TO WSI LEVEL** 

In [ ]:
# Obtaining WSI level label prediction using majority vote

wsi_true = {}
wsi_pred_counts = {}

# Group predictions by WSI and store true labels
for wsi, true_label, pred_label in zip(test_wsi_nos, y_true_test, y_pred_test):
    if wsi not in wsi_true:
        wsi_true[wsi] = true_label
    
    if wsi not in wsi_pred_counts:
        wsi_pred_counts[wsi] = {0: 0, 1: 0}
    
    wsi_pred_counts[wsi][pred_label] += 1
    
wsi_true_labels = []
wsi_pred_labels = []

for wsi, counts in wsi_pred_counts.items():
    wsi_true_labels.append(wsi_true[wsi])
    
    # Choose the label with the maximum count (majority vote)
    majority_vote = 0 if counts[0] > counts[1] else 1
    wsi_pred_labels.append(majority_vote)

In [ ]:
wsi_pred_counts 

In [ ]:
# Classification report for test set 

print("Test WSI Level Classification Report:\n")
print(classification_report(wsi_true_labels, wsi_pred_labels)) 

In [ ]:
# calculating CI 

accuracies = []
for i in range(1000):
    y_true_sample, y_pred_sample = resample(wsi_true_labels, wsi_pred_labels)
    accuracies.append(accuracy_score(y_true_sample, y_pred_sample))

# Compute confidence interval
alpha = 0.95
lower = np.percentile(accuracies, (1 - alpha) / 2 * 100)
upper = np.percentile(accuracies, (1 + alpha) / 2 * 100)

print(f"Accuracy: {np.mean(accuracies):.3f} with 95% CI: [{lower:.3f}, {upper:.3f}]") 

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(wsi_true_labels, wsi_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('WSI Level Confusion Matrix')
plt.show()